In [1]:
import pandas as pd

import os
from pathlib import Path

import sqlite3 as sql

### Establish Connection to uncleaned (raw) data:

In [2]:
project_path = Path(os.getcwd()).parent.absolute()
raw_data_path = project_path/'yify_collect_data'/'data'/'movie.db'

raw_conn = sql.connect(raw_data_path)
raw_cursor = raw_conn.cursor()

raw_db = pd.read_sql("SELECT * from movie", raw_conn)

# Clean raw database:

In [173]:
def get_unique_genre(genre_column):
    unique_genre = []
    for genres in genre_column:
        genres = genres.split('/')
        for genre in genres:
            cleaned_genre = genre.strip()
            if cleaned_genre not in unique_genre:
                unique_genre.append(cleaned_genre)
    return sorted(unique_genre)

### Better name for function
def remove_row(data, null_count=0):
    """Remove row, if contains the same amount or more null values than the null count"""
    data = data.copy(deep=True)
    drop_rows = data[data.isnull().sum(axis=1) >= null_count].index
    return data.drop(index=drop_rows)

df = raw_db.copy()

# Convert genre to a list of strings
df.genre = [genre.strip().split('/') for genre in df.genre]

# Expand genre such that each row has one genre only
df = df.explode('genre')

# Remove punctuation and Capatilazation
df.genre = [genre.replace('-', ' ').strip().lower() for genre in df.genre]

# Sorted unique genre values
unique_genre = get_unique_genre(df.genre)

# Map genre to dictionary
genre_dict = {key:value+1 for value, key in enumerate(unique_genre)}

# Apply map to dataframe
df.genre = [genre_dict[genre] for genre in df.genre]

# Update genre column name
df = df.rename({'genre':'genre_id'}, axis=1)

# Handle missing values
df = df.replace(['None'], None)
df = remove_row(df, null_count=1)

# Reset index
df = df.reset_index(drop=True)

# Remove movies that generated a 404 Error
df = df[df['title'] != '404 Error']

# Remove dubplicates
# df = df.drop_duplicates(['title', 'href'])

# Create new tables from cleaned df
movie = df.drop('genre_id', axis=1)
movie = movie.drop_duplicates(['title', 'href']).reset_index(drop=True)

movie_genre_map = df[['title', 'genre_id']]

genre = pd.DataFrame.from_dict({'genre_id':genre_dict.values(), 
                                'genre':genre_dict.keys()})

In [162]:
# Discovered duplicate title names 
## Create movie_id to track unique titles
temp = movie.copy()


temp[temp.duplicated(['title','year'], keep=False)].sort_values('title')

,title,year,rating,href
22422,15+ Coming of Age,2017,5.7,https://yts.mx/movies/15-coming-of-age-2017
15042,15+ Coming of Age,2017,5.7,https://yts.mx/movies/15-coming-of-age-2-2017
27822,A Mouse Tale,2012,4.9,https://yts.mx/movies/a-mouse-tale-2-2012
13859,A Mouse Tale,2012,4.9,https://yts.mx/movies/a-mouse-tale-2012
13784,Apartment 407,2016,5.8,https://yts.mx/movies/apartment-407-2016
...,...,...,...,...
33678,The Guardians,2017,6.7,https://yts.mx/movies/the-guardians-2017
7474,The Lovers' Guide: Igniting Desire,2011,6.2,https://yts.mx/movies/the-lovers-guide-ignitin...
27212,The Lovers' Guide: Igniting Desire,2011,0.0,https://yts.mx/movies/the-lovers-guide-ignitin...
11191,The Projectionist,2019,6.4,https://yts.mx/movies/the-projectionist-2019


---
# Create new database:

In [35]:
path = Path(os.getcwd())

conn = sql.connect(path/'movie.db')
cursor = conn.cursor()

In [164]:
# movie table
cursor.execute("""
    DROP TABLE IF EXISTS movie;
    """)

cursor.execute("""
    CREATE TABLE IF NOT EXISTS movie(
    title TEXT NOT NULL,
    year INTEGER,
    rating INT NOT NULL,
    href TEXT NOT NULL
    );""")

In [91]:
# movie_genre_map
cursor.execute("""
    DROP TABLE IF EXISTS movie_genre_map;
    """)

cursor.execute("""
    CREATE TABLE IF NOT EXISTS movie_genre_map(
    title TEXT NOT NULL,
    genre_id INTEGER,
    FOREIGN KEY(title) REFERENCES movie(title),
    FOREIGN KEY(genre_id) REFERENCES genre(genre_id)
    );""")

In [92]:
# genre table
cursor.execute("""
    DROP TABLE IF EXISTS genre;
    """)

cursor.execute("""
    CREATE TABLE IF NOT EXISTS genre(
    genre_id INTEGER PRIMARY KEY AUTOINCREMENT,
    genre TEXT NOT NULL
    );""")


### Inserting values into new database

In [93]:
# Have to first deal with duplicate titles
# movie.to_sql('movie', conn, if_exists='append', index=False)
genre.to_sql('genre', conn, if_exists='append', index=False)
movie_genre_map.to_sql('movie_genre_map', conn, if_exists='append', index=False)

140279